In [1]:
import numpy as np
import nltk as nl
from gensim.models import Word2Vec
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
surveys = pd.read_excel("C:\\Machine Learning\\Hackathons\\Xerox\\customer_Care_Survey_Refined.XLSX",header=0,sheet_name="Sheet2")

In [3]:
surveys.shape

(2315, 31)

In [4]:
surveys_comments = pd.DataFrame({"SR#":surveys["SR#"],
                                 "Comments":surveys["Do you have any other details you want to share regarding the ha"],
                                 "Additional_Comments":surveys["Please provide details#"],
                                 "Full_Comments":surveys["Full_Comments"],
                                 "Customer_SAT_Score":surveys["How satisfied are you with the way your issue was resolved?"]})

In [5]:
surveys.shape

(2315, 31)

In [6]:
surveys_comments.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
0,1-101526171125,Many thanks for understanding this issue and r...,NaN,Many thanks for understanding this issue and r...,10
1,1-101320199441,Thank you for everything David!,NaN,Thank you for everything David!,10
2,1-101571535327,We have been a customer of yours for many many...,No one wants to resolve this issue. Everyone ...,We have been a customer of yours for many many...,1
3,1-101428425708,It doesn't feel like a person helped me at all...,I needed information on a recurring bill we're...,It doesn't feel like a person helped me at all...,1
4,1-101292413714,none,NaN,none,10


In [7]:
surveys_comments["Full_Comments"][2], surveys_comments["Customer_SAT_Score"][2]

("We have been a customer of yours for many many years.  Technology and our needs have changed thus the request to end our contract.  You keep dodging a relatively small adjustment request which we should be owed; it's not a special request we are making.  This doesn't seem to me the way to respect a long time customer.  Please look into this without passing the buck to the next rep in line. No one wants to resolve this issue.  Everyone keeps passing the buck.",
 1)

In [8]:
surveys_comments.groupby(["Customer_SAT_Score"])["Customer_SAT_Score"].count()

Customer_SAT_Score
0       72
1      792
2       85
3       39
4       22
5       65
6       24
7       36
8       70
9      107
10    1003
Name: Customer_SAT_Score, dtype: int64

In [9]:
surveys_comments.loc[surveys_comments["Customer_SAT_Score"]<8,"Customer_SAT_Score"] = 0
surveys_comments.loc[surveys_comments["Customer_SAT_Score"]>=8,"Customer_SAT_Score"] = 1

In [10]:
surveys_comments

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
0,1-101526171125,Many thanks for understanding this issue and r...,NaN,Many thanks for understanding this issue and r...,1
1,1-101320199441,Thank you for everything David!,NaN,Thank you for everything David!,1
2,1-101571535327,We have been a customer of yours for many many...,No one wants to resolve this issue. Everyone ...,We have been a customer of yours for many many...,0
3,1-101428425708,It doesn't feel like a person helped me at all...,I needed information on a recurring bill we're...,It doesn't feel like a person helped me at all...,0
4,1-101292413714,none,NaN,none,1
5,1-101247040823,My original issue was handled in a timely mann...,There is still an issue with the fix that was ...,My original issue was handled in a timely mann...,0
6,1-101379044780,NaN,I got a form letter that did not properly ackn...,I got a form letter that did not properly ack...,0
7,1-101415299897,NaN,Communication with your customer service rep w...,Communication with your customer service rep ...,0
8,1-101344352092,NaN,My 700i was down for 6 weeks and all I ask for...,My 700i was down for 6 weeks and all I ask fo...,0
9,1-101366582438,The rep always acts with the most respect and ...,NaN,The rep always acts with the most respect and ...,1


In [11]:
from sklearn.model_selection import train_test_split
import os
from nltk.corpus import stopwords
import string
from collections import Counter

In [12]:
trainx_outer, testx_outer,trainy_outer, testy_outer = train_test_split(surveys_comments,
                                                                       surveys_comments["Customer_SAT_Score"],test_size=0.30,random_state =10,
                                                                       stratify = surveys_comments["Customer_SAT_Score"])


In [13]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    #print("table values are :\n")
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [14]:
trainx_outer.shape, testx_outer.shape,trainy_outer.shape, testy_outer.shape

((1620, 5), (695, 5), (1620,), (695,))

In [15]:
trainx_outer.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
496,1-99913066561,NaN,The issue was not resolved correctly. It took ...,The issue was not resolved correctly. It took...,0
55,1-101376461717,Just because one order was changed for billing...,NaN,Just because one order was changed for billing...,1
1055,1-98812644668,Susan provides efficient and timely service!,NaN,Susan provides efficient and timely service!,1
2056,1-95393097403,It is not the reps fault - he did nothing. It...,Xerox has not yet completed fixing the invoice...,It is not the reps fault - he did nothing. It...,0
2212,1-94645384791,The request to update the bill to address was ...,NaN,The request to update the bill to address was ...,1


In [16]:
testx_outer.head()

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score
1870,1-95729377061,thanks,NaN,thanks,1
1243,1-98294230571,Super detailed and helpful,NaN,Super detailed and helpful,1
1535,1-97220593657,great service thank you,NaN,great service thank you,1
1851,1-96315095579,Received invoice copies in a few days apprecia...,NaN,Received invoice copies in a few days apprecia...,1
2113,1-95335731026,This request was handled very quickly! Thank ...,NaN,This request was handled very quickly! Thank ...,1


In [17]:
trainx_outer["Full_Comments"].iloc[10]

'HELPFUL AND PLEASANT TO SPEAK WITH.  THANK YOU! '

In [18]:
vocab = Counter()
for i in range(len(trainx_outer)):
    tokens = clean_doc(trainx_outer["Full_Comments"].iloc[i])
    vocab.update(tokens)

In [19]:
len(vocab)

5024

In [20]:
print(vocab.most_common(5000))

[('Xerox', 425), ('The', 346), ('issue', 321), ('customer', 266), ('time', 252), ('service', 233), ('get', 233), ('request', 226), ('would', 222), ('invoice', 208), ('resolved', 207), ('invoices', 203), ('email', 200), ('still', 194), ('received', 192), ('call', 182), ('sent', 171), ('We', 166), ('This', 159), ('one', 155), ('It', 149), ('Thank', 149), ('machine', 147), ('account', 134), ('information', 131), ('address', 128), ('need', 126), ('rep', 126), ('response', 125), ('number', 118), ('told', 118), ('back', 113), ('billing', 110), ('requested', 110), ('credit', 108), ('contract', 107), ('know', 102), ('took', 100), ('never', 98), ('order', 97), ('us', 96), ('called', 95), ('late', 94), ('phone', 93), ('resolution', 91), ('contact', 87), ('person', 86), ('job', 85), ('problem', 84), ('someone', 84), ('representative', 83), ('resolve', 81), ('could', 79), ('provided', 78), ('needed', 77), ('new', 77), ('even', 77), ('since', 76), ('like', 76), ('name', 76), ('help', 74), ('work', 

In [21]:
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

2402


In [22]:
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()
save_list(tokens, 'vocab_CCO.txt')

In [23]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [24]:
# load the vocabulary
vocab_filename = 'vocab_CCO.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))

2402


In [25]:
def clean_doc_emb(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

In [26]:
records = list()
for i in range(len(trainx_outer)):
    tokens = clean_doc_emb(trainx_outer["Full_Comments"].iloc[i], vocab)
    records.append(tokens)

In [27]:
train_docs = records

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [29]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)

In [30]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [31]:
# pad sequences
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [32]:
Xtrain.shape

(1620, 252)

In [33]:
test_docs = list()
for i in range(len(testx_outer)):
    tokens = clean_doc_emb(testx_outer["Full_Comments"].iloc[i], vocab)
    test_docs.append(tokens)

In [34]:
encoded_docs_test = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [35]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM

In [36]:
vocab_size = len(tokenizer.word_index) + 1

## Pre Trained Model

In [37]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [38]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

In [39]:
os.chdir("C:\\Machine Learning\\Hackathons\\Sample Data\\NLP\\Sentiment-Analysis\\txt_sentoken\\")

In [40]:
from numpy import asarray
from numpy import zeros

In [41]:
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [42]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 252, 100)          208000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 124, 128)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15872)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 15873     
Total params: 288,001
Trainable params: 80,001
Non-trainable params: 208,000
_________________________________________________________________
None


In [43]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, trainy_outer, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, testy_outer, verbose=0)
print('Test Accuracy: {} and loss is {}' .format(acc*100,loss))

Epoch 1/10
 - 4s - loss: 0.5349 - acc: 0.7352
Epoch 2/10
 - 4s - loss: 0.3779 - acc: 0.8302
Epoch 3/10
 - 4s - loss: 0.2904 - acc: 0.8722
Epoch 4/10
 - 4s - loss: 0.2126 - acc: 0.9235
Epoch 5/10
 - 4s - loss: 0.1571 - acc: 0.9549
Epoch 6/10
 - 4s - loss: 0.1161 - acc: 0.9722
Epoch 7/10
 - 4s - loss: 0.0885 - acc: 0.9821
Epoch 8/10
 - 3s - loss: 0.0689 - acc: 0.9858
Epoch 9/10
 - 4s - loss: 0.0560 - acc: 0.9883
Epoch 10/10
 - 4s - loss: 0.0474 - acc: 0.9895
Test Accuracy: 82.30215821334784 and loss is 0.6067763826829924


In [44]:
y_predict = model.predict(Xtest)

In [45]:
y_predict[y_predict>=.5] = 1
y_predict[y_predict<.5] = 0

In [46]:
type(testy_outer.shape)

tuple

In [47]:
y_predict.shape

(695, 1)

In [48]:
final_comp =  pd.DataFrame(list(testy_outer), columns=["Actual"],)
print(final_comp.shape)
y_predict =   pd.DataFrame(y_predict, columns=["Predicted"],)
print(y_predict.shape)
print(testx_outer.shape)
y_predict.head()

(695, 1)
(695, 1)
(695, 5)


,Predicted
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [56]:
testtt = testx_outer
testtt.shape, testx_outer.shape

((695, 6), (695, 6))

In [57]:
temp = testtt
temp.reset_index(inplace=True,drop=True)
#temp.head()
act_vs_pred =pd.concat([ temp,y_predict], axis=1,)

In [59]:
act_vs_pred.drop(columns=["index"],inplace=True)

In [60]:
act_vs_pred

,SR#,Comments,Additional_Comments,Full_Comments,Customer_SAT_Score,Predicted
0,1-95729377061,thanks,NaN,thanks,1,1.0
1,1-98294230571,Super detailed and helpful,NaN,Super detailed and helpful,1,1.0
2,1-97220593657,great service thank you,NaN,great service thank you,1,1.0
3,1-96315095579,Received invoice copies in a few days apprecia...,NaN,Received invoice copies in a few days apprecia...,1,1.0
4,1-95335731026,This request was handled very quickly! Thank ...,NaN,This request was handled very quickly! Thank ...,1,1.0
5,1-99283726246,"He did an excellent job. Thanks, Connie",NaN,"He did an excellent job. Thanks, Conni...",1,1.0
6,1-99890763752,He was very professional!,NaN,He was very professional!,1,1.0
7,1-100711677606,I will not recommend anybody to buy Xerox prin...,It took over three weeks to resolve a simple p...,I will not recommend anybody to buy Xerox prin...,0,0.0
8,1-97652841926,All is good and thank you very much!,NaN,All is good and thank you very much!,1,1.0
9,1-99289237896,NaN,It didn’t tell me why they still have the mach...,It didn’t tell me why they still have the mac...,0,1.0


In [80]:
from sklearn.metrics import confusion_matrix

In [86]:
mat = confusion_matrix(act_vs_pred["Customer_SAT_Score"], act_vs_pred["Predicted"])

In [87]:
mat = pd.DataFrame(data=mat, columns=["Actual Positive","Actual Negative"],
                   index = ["Predicted Positive","Predicted Negative"])

In [88]:
mat

,Actual Positive,Actual Negative
Predicted Positive,296,45
Predicted Negative,78,276


In [63]:
296+276

572

In [64]:
78+45

123

In [66]:
572/695

0.823021582733813

## Sequencing Modeling